# MMF
* https://www.financialresearch.gov/money-market-funds/us-mmfs-investments-by-fund-category/
* Sources: SEC Form N-MFP2
* Six categories
 * Prime
 * Prime Institutional
 * Prime Retail
 * Government (No Fees & Gates)
 * Government
 * Government (Fees & Gates)
 * Tax Exempt
 * Tax Exempt Retail
 * Tax Exempt Institutional


In [1]:
import os
import json
import requests
import pandas as pd
from sqlalchemy import create_engine

response = requests.get('https://www.financialresearch.gov/money-market-funds/data/credit_risks_country/data.json')
data = json.loads(response.content)['datatable']['values']
df = pd.DataFrame(data, columns=['dt','tax_exempt_institutional', 'government_fees_gates', 'tax_exempt_retail', 'tax_exempt', 'prime_retail', 'prime_institutional', 'government', 'prime', 'government_no_fees_gates'])
df['total'] = df.iloc[:,1:].sum(axis=1)
df['dt'] = pd.to_datetime(df['dt'])
df['dt'] = df['dt'].dt.strftime('%Y%m%d')

sql_username = os.environ.get('SQL_USERNAME')
sql_password = os.environ.get('SQL_PASSWORD')
sql_host = os.environ.get('SQL_HOST')
sql_database = os.environ.get('SQL_DATABASE')
engine = create_engine(f'mysql://{sql_username}:{sql_password}@{sql_host}/{sql_database}')

tablename = 'mmf'
truncate_cmd = f'truncate table {tablename};'

with engine.connect() as conn:
    conn.execute(truncate_cmd)
    
df.to_sql('mmf', con=engine, if_exists='append', index=False)

146